# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770933736198                   -0.52    9.0         
  2   -2.772150594912       -2.91       -1.32    1.0    169ms
  3   -2.772170304555       -4.71       -2.45    1.0    139ms
  4   -2.772170653760       -6.46       -3.17    1.0    152ms
  5   -2.772170722835       -7.16       -4.26    2.0    175ms
  6   -2.772170722920      -10.07       -4.43    1.0    143ms
  7   -2.772170723012      -10.04       -5.47    1.0    160ms
  8   -2.772170723015      -11.56       -5.71    2.0    166ms
  9   -2.772170723015      -12.97       -6.31    1.0    157ms
 10   -2.772170723015   +  -15.05       -6.53    1.0    162ms
 11   -2.772170723015      -13.95       -7.20    1.0    160ms
 12   -2.772170723015   +    -Inf       -8.58    2.0    186ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735579752089763

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770766656091                   -0.52    9.0         
  2   -2.772060163387       -2.89       -1.32    1.0    137ms
  3   -2.772083036140       -4.64       -2.44    1.0    173ms
  4   -2.772083330778       -6.53       -3.12    1.0    140ms
  5   -2.772083416644       -7.07       -3.98    2.0    181ms
  6   -2.772083417568       -9.03       -4.40    1.0    166ms
  7   -2.772083417806       -9.62       -5.54    1.0    145ms
  8   -2.772083417810      -11.43       -5.75    2.0    181ms
  9   -2.772083417811      -12.36       -7.20    1.0    156ms
 10   -2.772083417811      -13.75       -7.06    3.0    201ms
 11   -2.772083417811   +  -13.94       -8.04    1.0    159ms

Polarizability via ForwardDiff:       1.7725349948979514
Polarizability via finite difference: 1.7735579752089763
